In [13]:
!apt install graphviz

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [10]:
import biosteam as bst
bst.nbtutorial() # For light-mode diagrams, ignore warnings

In [10]:
# Define chemicals used in the process
bst.settings.set_thermo(['Water', 'AceticAcid', 'EthylAcetate'])

# Amount of ethyl-acetate to fermentation broth
solvent_feed_ratio = 1.5
reflux = bst.Stream('reflux')
# Fermentation broth with dilute acetic acid
acetic_acid_broth = bst.Stream(ID='acetic_acid_broth', AceticAcid=1000, Water=9000, units='kg/hr')

# Solvent
ethyl_acetate = bst.Stream(ID='ethyl_acetate',  EthylAcetate=1)

# Products
glacial_acetic_acid = bst.Stream(ID='glacial_acetic_acid')
wastewater = bst.Stream(ID='wastewater')

# Recycles
solvent_recycle = bst.Stream('solvent_rich')
water_rich = bst.Stream('water_rich')
distillate = bst.Stream('raffinate_distillate')

# System and unit operations
with bst.System('AAsep') as sys:
    extractor = bst.MultiStageMixerSettlers(
        'extractor',
        ins=(acetic_acid_broth, ethyl_acetate, solvent_recycle),
        outs=('extract', 'raffinate'),
        top_chemical='EthylAcetate',
        feed_stages=(0, -1, -1),
        N_stages=12,
        use_cache=True,
    )

    @extractor.add_specification(run=True)
    def adjust_fresh_solvent_flow_rate():
        broth = acetic_acid_broth.F_mass
        EtAc_recycle = solvent_recycle.imass['EthylAcetate']
        EtAc_required = broth * solvent_feed_ratio
        if EtAc_required < EtAc_recycle:
            solvent_recycle.F_mass *= EtAc_required / EtAc_recycle
            EtAc_recycle = solvent_recycle.imass['EthylAcetate']
        EtAc_fresh = EtAc_required - EtAc_recycle
        ethyl_acetate.imass['EthylAcetate'] = max(
            0, EtAc_fresh
        )

    HX = bst.HXutility(
        'extract_heater',
        ins=(extractor.extract),
        outs=('hot_extract'),
        rigorous=True,
        V=0,
    )
    ED = bst.MESHDistillation(
                'extract_distiller',
                ins=(HX-0, reflux),
                outs=('', 'acetic_acid', 'distillate'),
                feed_stages=[7-2, 1],
                N_stages=9,
                full_condenser=True,
                boilup=4.48,
                LHK=('Water', 'AceticAcid'),
                use_cache=True,
            )

    ED2 = bst.MESHDistillation(
        'acetic_acid_purification',
        ins=ED-1,
        outs=('', glacial_acetic_acid),
        feed_stages=[7-2, 1],
        N_stages=9,
        full_condenser=False,
        boilup=4.48,
        LHK=('EthylAcetate', 'AceticAcid'),
        use_cache=True,
    )

    ED.check_LHK = ED2.check_LHK = False
    mixer = bst.Mixer(
        ins=(ED-0, ED2-0, distillate)
    )
    HX = bst.HXutility(ins=mixer-0, T=310)
    settler = bst.MixerSettler(
        'settler',
        ins=HX-0,
        outs=(solvent_recycle, water_rich),
        top_chemical='EthylAcetate',
    )
    mixer = bst.Mixer(ins=[extractor.raffinate, water_rich])
    RD = bst.ShortcutColumn(
        'raffinate_distiller',
        LHK=('EthylAcetate', 'Water'),
        ins=mixer-0,
        outs=[distillate, wastewater],
        partial_condenser=False,
        Lr=0.99,
        Hr=0.99,
        k=1.5,
    )

sys.simulate()
# sys.diagram(kind='cluster', format='png')
# sys.show()
sys.operating_hours = 330 * 24
print('CAPEX', round(sys.installed_equipment_cost / 1e6, 3), 'MMUSD')
print('OPEX', round(sys.material_cost + sys.utility_cost / 1e6, 4), 'MMUSD/yr')

CAPEX 1.166 MMUSD
OPEX 0.739 MMUSD/yr


In [3]:
sys.diagram(kind='thorough', file='diagram.png', format='png', display=False)

In [27]:
print("ED outputs:", ED.outs)  # ED의 출력 스트림 개수 확인


ED outputs: [<Stream: s3>, <Stream: acetic_acid>, <Stream: distillate>]


## ShortcutColumn 한개

In [41]:
# Define chemicals used in the process
bst.settings.set_thermo(['Water', 'AceticAcid'])

# Fermentation broth with dilute acetic acid
acetic_acid_broth = bst.Stream(ID='acetic_acid_broth', AceticAcid=1000, Water=9000, units='kg/hr')

# Products
glacial_acetic_acid = bst.Stream(ID='glacial_acetic_acid')
wastewater = bst.Stream(ID='wastewater')

# System and unit operations
with bst.System('AAsep') as sys:
    ED = bst.ShortcutColumn(
        'extract_distiller',
        ins=acetic_acid_broth,  # ED에 직접 연결
        outs=[wastewater, glacial_acetic_acid],  # 상부: Wastewater, 하부: Glacial Acetic Acid
        LHK=('Water', 'AceticAcid'),  # Water 제거, AceticAcid 농축
        Lr=0.99,  # AceticAcid의 하부 잔류 비율 증가
        Hr=0.99,  # Water의 상부 제거 비율 증가 (Water 완전히 제거)
        k=1.4,
        partial_condenser=False,
    )

sys.simulate()
sys.diagram(kind='cluster', format='png')
sys.show()
sys.operating_hours = 330 * 24
print('CAPEX', round(sys.installed_equipment_cost / 1e6, 3), 'MMUSD')
print('OPEX', round(sys.material_cost + sys.utility_cost / 1e6, 4), 'MMUSD/yr')


System: AAsep
ins...
[0] acetic_acid_broth  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water       500
                    AceticAcid  16.7
outs...
[0] wastewater  
    phase: 'l', T: 373.12 K, P: 101325 Pa
    flow (kmol/hr): Water       495
                    AceticAcid  0.167
[1] glacial_acetic_acid  
    phase: 'l', T: 380.62 K, P: 101325 Pa
    flow (kmol/hr): Water       5
                    AceticAcid  16.5
CAPEX 5.266 MMUSD
OPEX 8.0737 MMUSD/yr


In [36]:
sys.material_cost

0

In [37]:
sys.utility_cost


8073679.59104416

In [35]:
sys.unit_capital_costs.values()

AttributeError: 'System' object has no attribute 'unit_capital_costs'

In [33]:
ED.purchase_costs

{'Trays': 472495.3503163702,
 'Tower': 533432.8362885285,
 'Platform and ladders': 127721.13955939046,
 'Condenser - Floating head': 159711.25079586203,
 'Pump - Pump': 5888.663535150239,
 'Pump - Motor': 601.1805501058838,
 'Reboiler - Floating head': 87231.07143174125}

In [13]:
print(ED.results())
outlet = ED.reboiler.outs[0]
boilup = outlet['g'].F_mol / outlet['l'].F_mol
distillate, condensate = ED.top_split.outs
split = condensate.F_mol / ED.condenser.outs[0].F_mol # Or from ED.design_results['Reflux']
N_stages = int(ED.design_results['Theoretical stages'])
feed_stage = int(ED.design_results['Theoretical feed stage'])

Distillation Column                              Units  extract_distiller
Electricity         Power                           kW               56.2
                    Cost                        USD/hr                4.4
Cooling water       Duty                         kJ/hr          -2.07e+09
                    Flow                       kmol/hr           1.41e+06
                    Cost                        USD/hr                690
Low pressure steam  Duty                         kJ/hr           2.18e+09
                    Flow                       kmol/hr           5.64e+04
                    Cost                        USD/hr           1.34e+04
Design              Theoretical feed stage                            709
                    Theoretical stages                                761
                    Minimum reflux               Ratio               75.9
                    Reflux                       Ratio                106
                    Actual stages     

## 함수화 하여 결과 생성

In [1]:
cd ~/MultiFidelity-ProcessOpt/1.\ Code

/users/k23070952/MultiFidelity-ProcessOpt/1. Code


/users/k23070952/CODE/0. SSH files/jvenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/users/k23070952/CODE/0. SSH files/jvenv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import biosteam as bst
import pandas as pd
from OneColumn import OneColumn
from tqdm import tqdm 

# Load existing results dataframe
columns = [
    'Stages', '_Lr1', '_Hr1', '_Lr2', '_Hr2', '_T_hex', '_Lr3', '_Hr3',
    'Shortcut_CAPEX', 'Shortcut_OPEX', 'Shortcut_AceticAcidWt',
    'Shortcut_BoilupRatio', 'Shortcut_SplitRatio', 'Shortcut_NumberStages', 'Shortcut_FeedStage',
    'Rigorous_CAPEX', 'Rigorous_OPEX', 'Rigorous_AceticAcidWt'
]
results = pd.read_csv('/users/k23070952/MultiFidelity-ProcessOpt/1. Code/results.csv')
results_df = pd.DataFrame(results.values, columns=columns)

# Filter out invalid or NaN results for CAPEX
valid_results = results_df.dropna(subset=['Shortcut_AceticAcidWt', 'Rigorous_AceticAcidWt'])
valid_results = valid_results[(valid_results['Shortcut_AceticAcidWt'] > 0) & (valid_results['Rigorous_AceticAcidWt'] > 0) &
                                (valid_results['Shortcut_AceticAcidWt'] < 1) & (valid_results['Rigorous_AceticAcidWt'] < 1)]


# Compute CAPEX, OPEX, and Acetic Acid purity for each row
capex_list = []
opex_list = []
purity_list = []

for index, row in tqdm(results_df.iterrows()):
    Lr, Hr = row['_Lr1'], row['_Hr1']
    capex, opex, purity = OneColumn(Lr, Hr)
    capex_list.append(capex)
    opex_list.append(opex)
    purity_list.append(purity)

# Append new results to the dataframe
results_df['OneColumn_CAPEX'] = capex_list
results_df['OneColumn_OPEX'] = opex_list
results_df['OneColumn_AceticAcid_Purity'] = purity_list

# Save updated dataframe
results_df.to_csv('./OneColumn_results.csv', index=False)

0it [00:00, ?it/s]


RuntimeError: [38;2;229;63;229m<ShortcutColumn: extract_distiller>[0m failed to find bracket

In [23]:
valid_results = valid_results[(valid_results['_Lr1'] > 0.95) & (valid_results['_Hr1'] > 0.90) ]
valid_results

,Stages,_Lr1,_Hr1,_Lr2,_Hr2,...,Shortcut_NumberStages,Shortcut_FeedStage,Rigorous_CAPEX,Rigorous_OPEX,Rigorous_AceticAcidWt
195,10,0.986,0.938,0.203,0.339,...,12,8,1.28,1.39,0.313
469,39,0.951,0.963,0.863,0.567,...,10,8,2.3,5.96,0.509
822,42,0.997,0.996,0.52,0.77,...,19,14,1.6,1,0.789
832,13,0.997,0.978,0.61,0.267,...,16,10,1.79,2.64,0.659
900,33,0.975,0.98,0.962,0.408,...,11,8,1.74,1.5,0.78
1183,39,0.999,0.944,0.513,0.768,...,18,7,5.65,26.7,0.419
1454,25,0.985,0.959,0.626,0.224,...,12,8,2.23,6.63,0.344
1475,33,0.997,0.995,0.722,0.84,...,19,14,1.81,1.95,0.887
1523,37,0.99,0.928,0.415,0.348,...,13,8,1.34,1.26,0.471
